# MusciVAE

- input: 130차원(velocity) -> T, 9
- drum class: 9 canonical classes
- 16개의 note intervals -> 16 events
- 2-bar data -> T=32, 16-bar data -> T=256
- U = 16 -> 1개의 bar 안에 16개

- 2-bar
- input: T, 130, 9
- bar별로 16개의 note들
- rnn model: slide 1bar
- output: U=16

In [ ]:
"""
    Create a pandas to store the piano rolls in.
    It'll look a bit like this:
    
    |   Index     | t | C0 |...| C10 |
    |Song_name_3:0| 0 | 40 |...| 0   |
    |             | 1 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_3:1| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_4:0| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |

"""

In [4]:
import pandas as pd
import pretty_midi
import os
from getpass import getuser

# EDA

In [5]:
username = getuser()
base_path = f'/Users/{username}/data/midi/groove'
info_csv_filename = os.path.join(base_path, 'info.csv')

info_df = pd.read_csv(info_csv_filename)
info_df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test
...,...,...,...,...,...,...,...,...,...,...,...
1145,drummer2,drummer2/session2,drummer2/session2/11,rock,130,beat,4-4,drummer2/session2/11_rock_130_beat_4-4.mid,NaN,1.909613,train
1146,drummer2,drummer2/session2,drummer2/session2/12,rock,130,beat,4-4,drummer2/session2/12_rock_130_beat_4-4.mid,NaN,1.808652,train
1147,drummer2,drummer2/session2,drummer2/session2/13,rock,130,beat,4-4,drummer2/session2/13_rock_130_beat_4-4.mid,NaN,1.864421,train
1148,drummer2,drummer2/session2,drummer2/session2/14,rock,130,beat,4-4,drummer2/session2/14_rock_130_beat_4-4.mid,NaN,1.875960,train


In [9]:
sample_filename = info_df[info_df.split.eq('train')]['midi_filename'].iloc[0]
sample_filename = os.path.join(base_path, sample_filename)

midi = pretty_midi.PrettyMIDI(sample_filename)
ns = midi.instruments[0]

In [9]:
midi = pretty_midi.PrettyMIDI(sample_filename)
ns = midi.instruments[0]

In [ ]:
ROLAND_DRUM_PITCH_CLASSES = [
    # kick drum
    [36],
    # snare drum
    [38, 37, 40],
    # closed hi-hat
    [42, 22, 44],
    # open hi-hat
    [46, 26],
    # low tom
    [43, 58],
    # mid tom
    [47, 45],
    # high tom
    [50, 48],
    # crash cymbal
    [49, 52, 55, 57],
    # ride cymbal
    [51, 53, 59]
]

def _classes_to_map(classes):
      class_map = {}
      for cls, pitches in enumerate(classes):
        for pitch in pitches:
          class_map[pitch] = cls
      return class_map

def class_map():
    return _classes_to_map(ROLAND_DRUM_PITCH_CLASSES)

"""
    Create a pandas to store the piano rolls in.
    It'll look a bit like this:
    
    |   Index     | t | C0 |...| C10 |
    |Song_name_3:0| 0 | 40 |...| 0   |
    |             | 1 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_3:1| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |
    |Song_name_4:0| 0 | 40 |...| 0   |
    |             |...|... |...| ... |
    |             | N | 40 |...| 0   |

"""

In [27]:
len(ROLAND_DRUM_PITCH_CLASSES)
import numpy as np

midi = pretty_midi.PrettyMIDI(sample_filename)
ns = midi.instruments[0]

note_sequences = []
for i, note in enumerate(ns.notes):
    columns = np.zeros(len(ROLAND_DRUM_PITCH_CLASSES))
    columns[class_map()[note.pitch]] = note.velocity/127
    note_sequences.append(columns)

In [31]:
sample_filenames = info_df[info_df.split.eq('train')]['midi_filename']

note_dfs = []
for sample_filename in sample_filenames[:10]:
    sample_filename = os.path.join(base_path, sample_filename)
    midi = pretty_midi.PrettyMIDI(sample_filename)

    ns = midi.instruments[0]
    note_sequences = []
    for i, note in enumerate(ns.notes):
        columns = np.zeros(len(ROLAND_DRUM_PITCH_CLASSES))
        columns[class_map()[note.pitch]] = note.velocity/127
        note_sequences.append(columns)

    note_seq_df = pd.DataFrame(note_sequences)
    note_seq_df.reset_index(inplace=True)
    note_dfs.append(note_seq_df)

In [33]:
pd.concat(note_dfs, axis=0)

,index,0,1,2,3,4,5,6,7,8
0,0,0.000000,0.055118,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,1,0.000000,0.078740,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,2,0.000000,0.440945,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
3,3,0.000000,0.000000,0.511811,0.000000,0.0,0.0,0.0,0.0,0.000000
4,4,0.000000,0.377953,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
5,5,0.000000,0.212598,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
6,6,0.000000,0.000000,0.000000,0.330709,0.0,0.0,0.0,0.0,0.000000
7,7,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.480315
8,8,0.000000,0.236220,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000


In [1]:
from torch.utils.data import Dataset

class GrooveDataset(Dataset):
    def __init__(self, info_csv_filename, midi_dir, group='train'):
        info_df = pd.read_csv(info_csv_filename)
        self.midi_filenames = info_df[info_df.split.eq(group)]['midi_filename']
        self.midi_dir = midi_dir

    def __len__(self):
        return len(self.midi_filenames)

    def __getitem__(self, idx, ):
        item_filename = self.midi_filenames.iloc[idx]
        midi_filename = os.path.join(self.midi_dir, item_filename)
        midi = pretty_midi.PrettyMIDI(midi_filename)
        ns = midi.instruments[0]

        item = []
        for note in ns.notes:
            item.append([note.end-note.start, note.pitch, note.velocity])
        return item

In [2]:
base_path = '/Users/rimiiii/data/midi/groove'
info_csv_filename = os.path.join(base_path, 'info.csv')

groove_dataset = GrooveDataset(info_csv_filename, base_path)
groove_dataset[0]

NameError: name 'pd' is not defined

In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
# tf.enable_eager_execution()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

2022-12-05 23:34:24.715893: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2022-12-05 23:34:30.773410: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [30]:
print(dataset.batch(32))

<BatchDataset element_spec={'bpm': TensorSpec(shape=(None, None), dtype=tf.int32, name=None), 'drummer': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(None, None), dtype=tf.string, name=None), 'midi': TensorSpec(shape=(None, None), dtype=tf.string, name=None), 'style': {'primary': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'secondary': TensorSpec(shape=(None, None), dtype=tf.string, name=None)}, 'time_signature': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'type': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>
